### 데이터 병합시 클래스 번호 통일

- 두 데이터의 클래스 번호가 다르기 때문에, 같은 클래스는 같은 번호를 사용하도록 수정을 해줘야한다.

클래스 번호 변경 규칙:
- 첫 번째 데이터 세트 (`golf-ball,golf-club-handle,golf-club-head-1`):
- Golf Ball (0) → 그대로 유지
- Club Handle (1) → 2로 변경
- Club Head (2) → 1로 변경
- 두 번째 데이터 세트 (`galf-2`):
- Golf Ball (1) → 0으로 변경
- Club Head (0) → 1로 변경

- 최종 클래스 번호
    - Golf Ball - 0
    - Club Head - 1
    - Club Handle - 2

In [ ]:
import os

# 클래스 번호 매핑 규칙 정의
# 첫 번째 데이터 세트 클래스 매핑 (golf-ball,golf-club-handle,golf-club-head-1)
first_dataset_mapping = {0: 0, 1: 2, 2: 1}

# 두 번째 데이터 세트 클래스 매핑 (galf-2)
second_dataset_mapping = {0: 1, 1: 0}

def update_labels(file_path, class_mapping):
    """
    주어진 파일의 라벨링 번호를 class_mapping에 따라 업데이트하는 함수.
    
    Args:
        file_path (str): YOLO 형식 라벨링 파일 경로.
        class_mapping (dict): 기존 클래스 번호에서 새로운 번호로 매핑하는 딕셔너리.
    """
    with open(file_path, 'r') as file:
        lines = file.readlines()

    updated_lines = []
    for line in lines:
        components = line.strip().split()
        class_id = int(components[0])
        # 클래스 번호를 매핑 규칙에 따라 변환
        new_class_id = class_mapping.get(class_id, class_id)
        # 변환된 클래스 번호와 나머지 좌표 값들 유지
        updated_line = f"{new_class_id} " + " ".join(components[1:])
        updated_lines.append(updated_line)

    # 업데이트된 내용을 파일에 다시 저장
    with open(file_path, 'w') as file:
        file.write("\n".join(updated_lines) + "\n")

def process_labels_in_folders(base_folder_path, class_mapping):
    """
    주어진 폴더 내의 train, test, valid 하위 폴더의 labels 폴더 내 모든 YOLO 형식 라벨링 파일을 처리하는 함수.
    
    Args:
        base_folder_path (str): 데이터셋의 최상위 폴더 경로.
        class_mapping (dict): 기존 클래스 번호에서 새로운 번호로 매핑하는 딕셔너리.
    """
    subfolders = ['train', 'test', 'valid']
    
    for subfolder in subfolders:
        labels_folder = os.path.join(base_folder_path, subfolder, 'labels')
        
        if not os.path.exists(labels_folder):
            print(f"{labels_folder} 경로가 존재하지 않습니다.")
            continue
        
        for filename in os.listdir(labels_folder):
            if filename.endswith(".txt"):  # YOLO 형식 라벨링 파일만 처리
                file_path = os.path.join(labels_folder, filename)
                # print(f"Processing {file_path}...")
                update_labels(file_path, class_mapping)

# 첫 번째 데이터 세트 처리
process_labels_in_folders('./golf-ball,golf-club-handle,golf-club-head-1', first_dataset_mapping)

# 두 번째 데이터 세트 처리
process_labels_in_folders('./galf-2', second_dataset_mapping)

print("모든 파일이 성공적으로 업데이트되었습니다.")

### 데이터 병합

In [8]:
import os
import shutil

# 병합할 두 개의 폴더 경로
folder1 = './golf-ball,golf-club-handle,golf-club-head-1'
folder2 = './galf-2'

# 병합된 결과를 저장할 새로운 폴더 경로
merged_folder = 'merged_golfball_with_golf_club'

# 새로운 폴더 생성 (이미 존재하면 무시)
os.makedirs(merged_folder, exist_ok=True)

def merge_folders(src_folder, dest_folder):
    for root, dirs, files in os.walk(src_folder):
        # 현재 경로에서 상대 경로를 계산하여 대상 폴더에 복사
        relative_path = os.path.relpath(root, src_folder)
        dest_path = os.path.join(dest_folder, relative_path)

        # 대상 경로가 없으면 생성
        os.makedirs(dest_path, exist_ok=True)

        # 파일을 복사
        for file in files:
            src_file = os.path.join(root, file)
            dest_file = os.path.join(dest_path, file)
            shutil.copy2(src_file, dest_file)

# 첫 번째 폴더 병합
merge_folders(folder1, merged_folder)

# 두 번째 폴더 병합
merge_folders(folder2, merged_folder)

print(f"폴더가 성공적으로 {merged_folder}에 병합되었습니다.")


폴더가 성공적으로 merged_golfball_with_golf_club에 병합되었습니다.


### 데이터 개수 확인

In [10]:
import os

def count_images_in_folder(folder_path):
    image_count = {}
    # 이미지 파일 확장자 목록
    image_extensions = ('.jpg', '.jpeg', '.png', '.bmp', '.gif')

    # train, test, valid 폴더 각각에 대해 이미지 개수 세기
    for subset in ['train', 'valid', 'test']:
        subset_path = os.path.join(folder_path, subset, 'images')
        if os.path.exists(subset_path):
            # 해당 폴더 안의 이미지 파일 개수 세기
            image_files = [f for f in os.listdir(subset_path) if f.lower().endswith(image_extensions)]
            image_count[subset] = len(image_files)
        else:
            image_count[subset] = 0  # 해당 폴더가 없으면 0으로 설정
    return image_count

# 사용할 폴더 경로 (병합된 폴더 경로)
merged_folder_path = 'merged_golfball_with_golf_club'

# 이미지 개수 세기
image_counts = count_images_in_folder(merged_folder_path)

# 결과 출력
print("이미지 개수:")
for subset, count in image_counts.items():
    print(f"{subset}: {count}개")


이미지 개수:
train: 24867개
valid: 1904개
test: 1020개


### 클래스별 데이터 개수

In [3]:
import os

# 클래스 카운트를 저장할 딕셔너리 초기화
class_counts = {0: 0, 1: 0, 2: 0}

def count_classes_in_file(file_path):
    """
    주어진 YOLO 형식 라벨링 파일에서 각 클래스의 개수를 센다.
    
    Args:
        file_path (str): YOLO 형식 라벨링 파일 경로.
    """
    with open(file_path, 'r') as file:
        lines = file.readlines()

    for line in lines:
        parts = line.strip().split()
        if len(parts) == 0:
            continue  # 빈 라인 건너뛰기
        class_id = int(parts[0])  # 첫 번째 값이 클래스 ID
        if class_id in class_counts:
            class_counts[class_id] += 1

def process_labels_in_folders(base_folder_path):
    """
    주어진 폴더 내의 train, test, valid 하위 폴더의 labels 폴더 내 모든 YOLO 형식 라벨링 파일을 처리하여
    각 클래스의 개수를 센다.
    
    Args:
        base_folder_path (str): 데이터셋의 최상위 폴더 경로.
    """
    subfolders = ['train', 'test', 'valid']
    
    for subfolder in subfolders:
        labels_folder = os.path.join(base_folder_path, subfolder, 'labels')
        
        if not os.path.exists(labels_folder):
            print(f"{labels_folder} 경로가 존재하지 않습니다.")
            continue
        
        for filename in os.listdir(labels_folder):
            if filename.endswith(".txt"):  # YOLO 형식 라벨링 파일만 처리
                file_path = os.path.join(labels_folder, filename)
                # print(f"Processing {file_path}...")
                count_classes_in_file(file_path)

# 병합된 데이터 세트 처리
process_labels_in_folders('./merged_golfball_with_golf_club')

# 결과 출력
print("클래스별 데이터 개수")
print(f"Golf Ball (0): {class_counts[0]}개")
print(f"Club Head (1): {class_counts[1]}개")
print(f"Club Handle (2): {class_counts[2]}개")

클래스별 데이터 개수
Golf Ball (0): 24077개
Club Head (1): 21725개
Club Handle (2): 12193개
